# Digital Tools for Finance - Project
**Research Question:**\
What is the effect of the U.S. GDP growth rate on different S&P500 sector indices' market capitalizations\

**Authors:**\
Jonas Neller, 21-730-676\
Lorena Tassone, 18-700-237\
Naomi Huser, 17-056-201

## ToDo List:
API einrichten mit FRED\
Up- and downstream functions for database\
<s>Data cleaning</s> -> NaN values?\
<s>Regressions</s>  -> which value to regress?\
plots

create docker container for jupyter-notebook and app


--> interactive app -> Use ipywidget for app of Jupyter Notebook! \
--> good comments in code and github commits! \
--> Fore presentation use beamer!




## Data Cleaning

In [13]:
# libraries
import numpy as np
import pandas as pd
import os
from functools import reduce
import statsmodels.formula.api as smf 
from statsmodels.iolib.summary2 import summary_col

from Digital_Tools_for_Finance.data.interim import downstream


#from connect import DATAPATH
# import functions


ModuleNotFoundError: No module named 'Digital_Tools_for_Finance'

In [ ]:
# get mcap data
data_mcap
# later changed to mio.

# get macro data for each factor
data_gdp = get_macro_data('Real_GDP', "2007-01-01", "2021-01-01") # in bio.
data_gdp['Real_GDP'] = data_gdp['Real_GDP'] * 1000 # now in mio
data_unempr = get_macro_data('Unemployment_rate', "2007-01-01", "2021-01-01")
data_crudeprice = get_macro_data('Crude_price', "2007-01-01", "2021-01-01")
data_feddebt = get_macro_data('Federal_debt', "2007-01-01", "2021-01-01") # in mio.
data_fedrate = get_macro_data('Federal_rate', "2007-01-01", "2021-01-01")



In [6]:
# create dataframes
data_mcap = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Market_Cap') # later changed to mio.
data_gdp = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Real_GDP').set_index('Date ') # in bio.
data_gdp['Real_GDP'] = data_gdp['Real_GDP'] * 1000 # now in mio
data_unempr = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Unemployment_rate').set_index('Date ')
data_crudeprice = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Crude_price').set_index('Date ')
data_feddebt = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Federal_debt').set_index('Date ') # in mio.
data_fedrate = pd.read_excel('../data/processed/Data.xlsx', sheet_name='Federal_rate').set_index('Date ')

data_mcap = data_mcap.transpose()
data_mcap.columns = data_mcap.loc['Sector',:]


# create sector subsets
sectors = data_mcap.loc['Sector',:].unique()
print(sectors)

communication = data_mcap[['Communication Services']]
consumerdisc = data_mcap[['Consumer Discretionary']] 
consumerstap = data_mcap[['Consumer Staples']] 
energy = data_mcap[['Energy']] 
financials = data_mcap[['Financials']] 
healthcare = data_mcap[['Health Care']] 
industrials = data_mcap[['Industrials']] 
informationtech = data_mcap[['Information Technology']] 
materials = data_mcap[['Materials']] 
realestate = data_mcap[['Real Estate']] 
utilities = data_mcap[['Utilities']] 

print(data_mcap.loc["2006-12-29":,"Communication Services"])

['Communication Services' 'Consumer Discretionary' 'Consumer Staples'
 'Energy' 'Financials' 'Health Care' 'Industrials'
 'Information Technology' 'Materials' 'Real Estate' 'Utilities']


KeyError: '2006-12-29'

In [9]:
# merge factors by quarterly date
data_frames = [data_gdp, data_unempr, data_crudeprice, data_feddebt, data_fedrate]
data_factors_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date '], how='inner'), data_frames)
print(data_factors_merged.loc["2007-01-01":"2021-01-01",:])

              Real_GDP  Delta_real_GDP  Unemployment_rate  Crude_price  \
Date                                                                     
2007-01-01  15478956.0             NaN                4.6    54.780000   
2007-04-01  15577779.0        1.006384                4.5    63.974762   
2007-07-01  15671605.0        1.006023                4.7    74.046364   
2007-10-01  15767146.0        1.006096                4.7    85.926522   
2008-01-01  15702906.0        0.995926                5.0    92.982609   
2008-04-01  15792773.0        1.005723                5.0   112.336364   
2008-07-01  15709562.0        0.994731                5.8   133.958261   
2008-10-01  15366607.0        0.978169                6.5    76.283913   
2009-01-01  15187475.0        0.988343                7.8    41.630909   
2009-04-01  15161772.0        0.998308                9.0    49.905909   
2009-07-01  15216647.0        1.003619                9.5    64.208696   
2009-10-01  15379155.0        1.010680

In [32]:
# checking timeframe length
print(len(utilities.index)-3)
print(len(data_factors_merged))

57
57


In [33]:
#Drop Companies with missing Market Cap Data
communication = communication.dropna(axis=1)
communication = communication.tail(-3)
communication.columns = np.arange(len(communication.columns))
communication = communication.set_index(data_factors_merged.index)
communication = communication/1000000

consumerdisc = consumerdisc.dropna(axis=1)
consumerdisc = consumerdisc.tail(-3)
consumerdisc.columns = np.arange(len(consumerdisc.columns))
consumerdisc = consumerdisc.set_index(data_factors_merged.index)
consumerdisc = consumerdisc/1000000

consumerstap = consumerstap.dropna(axis=1)
consumerstap = consumerstap.tail(-3)
consumerstap.columns = np.arange(len(consumerstap.columns))
consumerstap = consumerstap.set_index(data_factors_merged.index)
consumerstap = consumerstap/1000000

energy = energy.dropna(axis=1)
energy = energy.tail(-3)
energy.columns = np.arange(len(energy.columns))
energy = energy.set_index(data_factors_merged.index)
energy = energy/1000000

financials = financials.dropna(axis=1)
financials = financials.tail(-3)
financials.columns = np.arange(len(financials.columns))
financials = financials.set_index(data_factors_merged.index)
financials = financials/1000000

healthcare = healthcare.dropna(axis=1)
healthcare = healthcare.tail(-3)
healthcare.columns = np.arange(len(healthcare.columns))
healthcare = healthcare.set_index(data_factors_merged.index)
healthcare = healthcare/1000000

industrials = industrials.dropna(axis=1)
industrials = industrials.tail(-3)
industrials.columns = np.arange(len(industrials.columns))
industrials = industrials.set_index(data_factors_merged.index)
industrials = industrials/1000000

informationtech = informationtech.dropna(axis=1)
informationtech = informationtech.tail(-3)
informationtech.columns = np.arange(len(informationtech.columns))
informationtech = informationtech.set_index(data_factors_merged.index)
informationtech = informationtech/1000000

materials = materials.dropna(axis=1)
materials = materials.tail(-3)
materials.columns = np.arange(len(materials.columns))
materials = materials.set_index(data_factors_merged.index)
materials = materials/1000000

realestate = realestate.dropna(axis=1)
realestate = realestate.tail(-3)
realestate.columns = np.arange(len(realestate.columns))
realestate = realestate.set_index(data_factors_merged.index)
realestate = realestate/1000000

utilities = utilities.dropna(axis=1)
utilities = utilities.tail(-3)
utilities.columns = np.arange(len(utilities.columns))
utilities = utilities.set_index(data_factors_merged.index)
utilities = utilities/1000000

utilities

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
Date,,,,,,,,,,,,,,,,,,,,,
2007-01-01,14638.712578,4384.709584,11082.465319,16843.471728,2783.812956,5186.872195,3714.65929,12344.091522,29655.753926,7330.301892,...,2585.436457,7082.069181,16509.186009,5042.522737,12668.866168,16741.258572,14692.202604,27377.404804,5551.817787,9382.708997
2007-04-01,14293.334605,5203.142484,10374.986145,19337.89338,2770.689248,5742.217475,3973.954379,13155.459854,31087.839704,7218.796736,...,2630.056061,8813.046971,16933.949864,4799.79724,14457.495295,20942.966433,16040.666104,27435.978163,5674.427246,10067.390535
2007-07-01,14619.747943,4510.086515,10170.069511,17960.461536,2669.515424,5581.703213,3840.000861,12205.768099,30235.899196,7223.576367,...,2230.39886,10073.112452,15922.900532,3964.184871,16710.462346,22263.327148,15625.445214,25779.517271,5172.710575,8370.499773
2007-10-01,13399.013097,4448.559078,10899.08568,18395.319997,2525.014004,5148.532071,3777.497632,12546.154677,29412.699509,7033.349506,...,2062.030362,10142.537781,16890.508337,3963.216018,16457.065022,22374.390647,15345.081971,27443.19155,5266.183705,9044.078163
2008-01-01,14315.413925,4723.69187,11276.176509,18643.867338,2516.58313,5503.085217,3903.264497,13237.141598,27305.014148,6123.4359,...,2035.175796,10343.243513,15256.086159,4257.329377,17988.1122,24983.726705,16176.116207,29429.803539,5696.297037,9477.82186
2008-04-01,11189.927438,3866.971305,9197.420441,16669.763688,2288.618753,4684.342635,3048.897541,10803.922469,23605.367136,5398.508272,...,1857.445108,9218.884263,13123.3939,3521.507063,15877.949067,20441.693328,13922.387942,27231.399982,5143.660652,8561.502181
2008-07-01,12905.493651,3783.440279,8819.41565,16156.006131,2486.498186,5268.65447,3355.138358,10657.32017,27541.250111,5889.372638,...,1930.919694,10121.052813,14179.609591,3088.847558,18091.052074,23361.208623,14131.787159,26894.940376,5287.119732,8647.303241
2008-10-01,7864.423442,3557.519311,8204.430691,14895.645178,2412.504636,4980.371192,2812.043123,11736.279603,24869.781498,5565.046732,...,1590.548582,5840.619073,13427.923773,3466.242144,12862.862455,16677.858279,12434.716283,29193.933806,5249.66292,8615.777616
2009-01-01,5456.734733,3222.862738,7061.098,13514.0514,2169.898791,4343.307958,2286.958674,10652.401729,20835.272765,4947.609526,...,1182.276793,5467.542208,13973.688046,3239.171909,10665.887123,14762.794155,10385.975974,28786.0,4908.259452,8329.858931


In [34]:
# take average mcap of each sector
data_factors_merged['mean_communication'] = communication.mean(axis=1)
data_factors_merged['mean_consumerdisc'] = consumerdisc.mean(axis=1)
data_factors_merged['mean_consumerstap'] = consumerstap.mean(axis=1)
data_factors_merged['mean_energy'] = energy.mean(axis=1)
data_factors_merged['mean_financials'] = financials.mean(axis=1)
data_factors_merged['mean_healthcare'] = healthcare.mean(axis=1)
data_factors_merged['mean_industrials'] = industrials.mean(axis=1)
data_factors_merged['mean_informationtech'] = informationtech.mean(axis=1)
data_factors_merged['mean_materials'] = materials.mean(axis=1)
data_factors_merged['mean_realestate'] = realestate.mean(axis=1)
data_factors_merged['mean_utilities'] = utilities.mean(axis=1)
data_factors_merged



,Real_GDP,Delta_real_GDP,Unemployment_rate,Crude_price,Federal_debt,Delta_Fed_Debt,Federal_rate,mean_communication,mean_consumerdisc,mean_consumerstap,mean_energy,mean_financials,mean_healthcare,mean_industrials,mean_informationtech,mean_materials,mean_realestate,mean_utilities
Date,,,,,,,,,,,,,,,,,,
2007-01-01,15478956.0,NaN,4.6,54.780000,8849665,NaN,5.25,33566.840117,13976.755474,35114.892851,56310.751253,37776.126421,22132.352255,20349.909097,27144.892828,7853.160657,7264.136631,12883.127893
2007-04-01,15577779.0,1.006384,4.5,63.974762,8867677,1.002035,5.25,38302.249478,14503.530138,35212.858725,56438.893339,37015.969139,22413.011674,20382.270076,26421.560145,9070.796753,7742.640188,13911.226269
2007-07-01,15671605.0,1.006023,4.7,74.046364,9007653,1.015785,5.26,40624.381397,14662.968567,35588.168359,63317.858592,37527.513582,23103.166518,22189.265586,29391.742272,9796.573312,7148.311071,13639.476060
2007-10-01,15767146.0,1.006096,4.7,85.926522,9229172,1.024592,4.76,41623.296464,15037.524626,36732.824113,69278.232520,37453.745195,23375.133972,23358.406261,31306.212811,10566.540863,7338.628824,13798.571143
2008-01-01,15702906.0,0.995926,5.0,92.982609,9437594,1.022583,3.94,42096.907689,13285.795094,37801.508990,70566.237756,35004.099575,23350.162683,21849.902373,31631.543364,11177.673054,6721.944800,14455.568098
2008-04-01,15792773.0,1.005723,5.0,112.336364,9492006,1.005765,2.28,34650.526560,12018.632715,37128.022826,63541.830952,31040.045174,20347.138196,20693.358493,26651.761460,10757.432186,6579.862658,12858.594423
2008-07-01,15709562.0,0.994731,5.8,133.958261,10024725,1.056123,2.01,34676.869800,10568.747299,34485.101579,71802.664608,26359.737511,19646.907437,18369.901164,27269.837431,12293.842884,6345.412886,13743.757261
2008-10-01,15366607.0,0.978169,6.5,76.283913,10699805,1.067341,0.97,30124.157814,10736.198514,36335.700442,56421.450117,29074.541413,19459.001754,16690.777128,24108.561465,8464.278004,6663.310598,11613.945559
2009-01-01,15187475.0,0.988343,7.8,41.630909,11126941,1.039920,0.15,25669.455070,8256.185435,31451.094681,47308.732096,19784.217875,16791.544394,12571.210003,17901.993943,6054.703379,4424.134370,10315.682971


## Regression

In [38]:
# Regress mean market cap on GDP growth and controlling variables for all sectors
list_of_sectors = ["mean_communication","mean_consumerdisc","mean_consumerstap","mean_energy","mean_financials","mean_healthcare","mean_industrials","mean_informationtech","mean_materials","mean_realestate","mean_utilities"]

# regressions & output
models = []

for resp in list_of_sectors:
    formula = resp + " ~ Delta_real_GDP + Unemployment_rate + Crude_price + Delta_Fed_Debt + Federal_rate"
    res = smf.ols(formula, data = data_factors_merged).fit()
    models.append(res)

regoutputs = summary_col(models,stars=True)
print(regoutputs)

# latex table for paper
regoutputs.as_latex()
    


                  mean_communication mean_consumerdisc mean_consumerstap  mean_energy   mean_financials mean_healthcare mean_industrials mean_informationtech mean_materials mean_realestate mean_utilities
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept         -1041106.7060      -333665.7452      -106885.6519      290941.8849**  -63586.6471     -218590.9776    -64715.8475      -846475.2741*        -64031.3793    -60112.3272     -69659.6669   
                  (729703.5645)      (261477.9634)     (167146.4049)     (111141.8835)  (177514.8398)   (202409.9667)   (105979.1253)    (502641.2050)        (64389.8874)   (92808.7157)    (84719.7386)  
Delta_real_GDP    647835.0304        238470.3963       91029.8197        -148100.3385** 69803.6646      151079.8944     57393.1973       551020.5982*         47541.0319     48261.5923

'\\begin{table}\n\\caption{}\n\\label{}\n\\begin{center}\n\\begin{tabular}{llllllllllll}\n\\hline\n                   & mean\\_communication & mean\\_consumerdisc & mean\\_consumerstap & mean\\_energy   & mean\\_financials & mean\\_healthcare & mean\\_industrials & mean\\_informationtech & mean\\_materials & mean\\_realestate & mean\\_utilities  \\\\\n\\hline\nIntercept          & -1041106.7060       & -333665.7452       & -106885.6519       & 290941.8849**  & -63586.6471      & -218590.9776     & -64715.8475       & -846475.2741*         & -64031.3793     & -60112.3272      & -69659.6669      \\\\\n                   & (729703.5645)       & (261477.9634)      & (167146.4049)      & (111141.8835)  & (177514.8398)    & (202409.9667)    & (105979.1253)     & (502641.2050)         & (64389.8874)    & (92808.7157)     & (84719.7386)     \\\\\nDelta\\_real\\_GDP   & 647835.0304         & 238470.3963        & 91029.8197         & -148100.3385** & 69803.6646       & 151079.8944      & 57393.1